In [19]:
import os, json, pickle, csv, pandas
import numpy as np
from classDeclarations import file_data
from random import sample
MAX_SAMPLE_SIZE = 100

In [20]:
Path = "../../CDIACPub8"

imgs = []
csvs = []
pdfs = []

missing_count = 0
df = pandas.read_csv('../CDIACFileData/labels/cdiac_naivetruth_processed.csv')
for subdir, dirs, files in os.walk(Path):
    for file_name in files:
        file_path = os.path.abspath(os.path.join(subdir, file_name))
        if df['path'].str.contains(file_name).any():
            if file_path.endswith('.png') or file_path.endswith('.jpg'):
                imgs.append(file_path)
            elif file_path.endswith('.csv') or file_path.endswith('.tsv'):
                csvs.append(file_path)
            elif file_path.endswith('.pdf') or file_path.endswith('.txt'):
                pdfs.append(file_path)
        else:
            missing_count += 1


img_sample_names = sample(imgs, MAX_SAMPLE_SIZE)
csv_sample_names = sample(csvs, MAX_SAMPLE_SIZE)
pdfs_sample_names = sample(pdfs, MAX_SAMPLE_SIZE)

/home/cc/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [21]:
print("loading files now...")
one_gram = json.load(open('../CDIACFileData/one_grams_distr_256.json', 'r'))
two_gram = json.load(open('../CDIACFileData/two_grams_distr_256.json', 'r'))

byte_distr = pandas.DataFrame.from_dict(one_gram)
two_grams_dicts = pandas.DataFrame.from_dict(two_gram)
#best_extractors = json.load(open("../CorrelatingExtractors/best_extractors.json"))
print("loading files done!")

loading files now...
loading files done!


In [22]:
byte_distr_new_index = byte_distr.index.tolist()
byte_distr_new_index.append("file_type")
byte_distr_new_index.append("file_size")
byte_distr = byte_distr.reindex(byte_distr_new_index)

two_grams_dicts_new_index = two_grams_dicts.index.tolist()
two_grams_dicts_new_index.append("file_type")
two_grams_dicts_new_index.append("file_size")
two_grams_dicts = two_grams_dicts.reindex(two_grams_dicts_new_index)

#byte_distr.sort_index(key=lambda x: x.str.lower())
#two_grams_dicts.sort_index(key=lambda x: x.str.lower())

In [23]:
img_data = []
csv_data = []
pdf_data = []
dataset = [img_data, csv_data, pdf_data]
for idx, data_list in enumerate(dataset):
    curr_sample = []
    if idx == 0:
        curr_sample = img_sample_names
    elif idx == 1:
        curr_sample = csv_sample_names
    else:
        curr_sample = pdfs_sample_names
    print("Done with index", idx)

    

    for file_name in curr_sample:
        #best_extractor_row_index = df.path[df.path==file_name].index.tolist()

        if  file_name.endswith('.txt') or file_name.endswith('.pdf'):
            best_extractor = 'freetext'
            byte_distr.at['file_type', file_name] = 0 
            two_grams_dicts.at['file_type', file_name] = 0
        elif file_name.endswith('.csv') or file_name.endswith('.tsv'):
            best_extractor = 'tabular'
            byte_distr.at['file_type', file_name] = 1
            two_grams_dicts.at['file_type', file_name] = 1
        elif file_name.endswith('.png') or file_name.endswith('.jpg'):
            best_extractor = 'image'
            byte_distr.at['file_type', file_name] = 2
            two_grams_dicts.at['file_type', file_name] = 2
          
        byte_distr.at['file_size', file_name] = os.path.getsize(file_name)
        two_grams_dicts.at['file_size', file_name] = os.path.getsize(file_name)

        curr_file_data = file_data(file_name, byte_distr[file_name], two_grams_dicts[file_name], best_extractor)
        data_list.append(curr_file_data)

Done with index 0
Done with index 1
Done with index 2


In [24]:
print('Dumping!')

with open('gathered_data_appened_256KB_v1.pkl', 'wb+') as handle:
    pickle.dump(dataset, file=handle, protocol=pickle.HIGHEST_PROTOCOL)
print('Dumped.')



Dumping!
Dumped.
